In [1]:
!git clone https://github.com/openai/CLIP.git
%cd /content/CLIP/
!pip install ftfy
import clip
%cd /content

Cloning into 'CLIP'...
remote: Enumerating objects: 256, done.
remote: Total 256 (delta 0), reused 0 (delta 0), pack-reused 256 (from 1)
Receiving objects: 100% (256/256), 8.87 MiB | 26.25 MiB/s, done.
Resolving deltas: 100% (136/136), done.
/content/CLIP
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
/content


In [2]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# Load clip
perceptor, preprocess = clip.load('ViT-B/32')
perceptor = perceptor.to(device)

100%|███████████████████████████████████████| 338M/338M [00:03<00:00, 95.4MiB/s]


In [4]:
# encode the text into the CLIP latent space using this code
text = "Genshin-style character"
tx = clip.tokenize(text).to(device)
E_text = perceptor.encode_text(tx)

In [5]:
# from lab03
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

In [6]:
!pip install huggingface_hub

In [7]:
from huggingface_hub import hf_hub_download

weights_path = hf_hub_download(
    repo_id="lfw3/Genshin_Stats_MLP_to_CLIP",
    filename="best_clip_offset_mlp.pt"
)

# mlp.py contains the model architecture (class definition)
code_path = hf_hub_download(
    repo_id="lfw3/Genshin_Stats_MLP_to_CLIP",
    filename="clip_mlp.py"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


best_clip_offset_mlp.pt:   0%|          | 0.00/41.3M [00:00<?, ?B/s]

clip_mlp.py: 0.00B [00:00, ?B/s]

In [9]:
import importlib.util
spec = importlib.util.spec_from_file_location("mlp", code_path)
mlp_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(mlp_module)

MLP = mlp_module.CLIPEmbeddingMLP

In [11]:
import torch

model = MLP()
checkpoint = torch.load(weights_path, map_location="cpu")
# state = torch.load(weights_path, map_location="cpu")
state = checkpoint["model_state_dict"]
model.load_state_dict(state)
model.to("cuda")
model.eval()

RuntimeError: Error(s) in loading state_dict for CLIPEmbeddingMLP:
	size mismatch for mlp.0.weight: copying a param with shape torch.Size([1024, 1560]) from checkpoint, the shape in current model is torch.Size([1024, 1556]).

In [ ]:
newData = []
E_offset = model(newData)

In [ ]:
# prepare the embedding
E_star = E_text + E_offset

# Load the diffuser model


In [12]:
%%capture
!pip install -U peft transformers diffusers

In [13]:
from diffusers import DiffusionPipeline
import torch

# Load Stable Diffusion XL Base1.0
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

# Optional CPU offloading to save some GPU Memory
pipe.enable_model_cpu_offload()

# Loading Trained LoRA Weights
# TODO: change it to the path point to our weights.
# Can be a path to a local folder, or a huggingface model hub repo
pipe.load_lora_weights("mary-ruiliii/genshin-style_character_generator")

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

vae_1_0/diffusion_pytorch_model.fp16.saf(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_lora_weights.safetensors:   0%|          | 0.00/23.4M [00:00<?, ?B/s]

No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
No LoRA keys associated to CLIPTextModelWithProjection found with the prefix='text_encoder_2'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModelWithProjection related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


In [2]:
# # Our own embeddings
# my_prompt_embeds = E_star.half().cuda()
# my_pooled_embeds = "Genshin-style character"

# # Invoke pipeline to generate image
# image = pipe(
#     prompt_embeds=my_prompt_embeds,
#     pooled_prompt_embeds=my_pooled_embeds,
#     num_inference_steps=50,
#     guidance_scale=7.0,
#     height=1024,
#     width=1024
# ).images[0]
prompt = "Genshin-style character"

# Invoke pipeline to generate image
image = pipe(
    prompt = prompt,
    num_inference_steps=50,
    height=400,
    width=400,
    guidance_scale=7.0,
).images[0]

NameError: name 'pipe' is not defined

In [1]:
image

NameError: name 'image' is not defined

In [16]:
# Save Image
image.save(f"sdxl_oldbookillustrations_{prompt}.png")

# Something else

In [ ]:
def precision(model, dataset):
    """
    Compute the precision of `model` over the `dataset`.  We will take the
    **most probable class** as the class predicted by the model.

    Parameters:
        `model` - A PyTorch MLPModel
        `dataset` - A data structure that acts like a list of 2-tuples of
                  the form (x, t), where `x` is a PyTorch tensor of shape
                  [1, 28, 28] representing an MedMNIST image,
                  and `t` is the corresponding binary target label

    Returns: a floating-point value between 0 and 1.
    """
    true_pos, total_pred_pos = 0, 0
    loader = torch.utils.data.DataLoader(dataset, batch_size=100)
    for img, t in loader:
        X = img.reshape(-1, 784)
        z = model(X)

        y = torch.sigmoid(z) > 0.5 # TODO: pred should be a [N, 1] tensor with binary
                    # predictions, (0 or 1 in each entry)

        yt = y + t
        true_pos += int(torch.sum(yt == 2))
        total_pred_pos += int(torch.sum(y == 1))

        # TODO: update total_pred_pos and true_pos
    return true_pos / total_pred_pos


def recall(model, dataset):
    """
    Compute the recall (or sensitivity) of `model` over the `dataset`.  We will
    take the **most probable class** as the class predicted by the model.

    Parameters:
        `model` - A PyTorch MLPModel
        `dataset` - A data structure that acts like a list of 2-tuples of
                  the form (x, t), where `x` is a PyTorch tensor of shape
                  [1, 28, 28] representing an MedMNIST image,
                  and `t` is the corresponding binary target label

    Returns: a floating-point value between 0 and 1.
    """
    true_pos, total_actual_pos = 0, 0 # track the true and false positive
    loader = torch.utils.data.DataLoader(dataset, batch_size=100)
    for img, t in loader:
        X = img.reshape(-1, 784)
        z = model(X)

        y = torch.sigmoid(z) > 0.5 # TODO: pred should be a [N, 1] tensor with binary
                    # predictions, (0 or 1 in each entry)

        yt = y + t
        true_pos += int(torch.sum(yt == 2))
        total_actual_pos += int(torch.sum(t == 1))

        # TODO: update total_pos and true_pos
    return true_pos / total_actual_pos

print("Precision(Training)", precision(model, train_data))
print("Recall(Training)", recall(model, train_data))
print("Precision(Validation)", precision(model, val_data))
print("Recall(Validation)", recall(model, val_data))

In [ ]:
m_once = MLPModel()
train_model(m_once, train_data, val_data, learning_rate=0.5, batch_size=500, num_epochs=1)
print("Training Accuracy:", accuracy(m_once, train_data))
print("Validation Accuracy:", accuracy(m_once, val_data))
z = model(X.view(-1, 784))
y = torch.sigmoid(z)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def get_prediction(model, data, sample=1000):
    loader = torch.utils.data.DataLoader(data, batch_size=sample, shuffle=True)
    for X, t in loader:
        z = model(X.view(-1, 784))
        y = torch.sigmoid(z)
        break
    y = y.detach().numpy()
    t = t.detach().numpy()
    return y, t

y, t = get_prediction(model, val_data)
y = y > 0.5
cm = confusion_matrix(t, y)
cmp = ConfusionMatrixDisplay(cm, display_labels=["0", "1"])
cmp.plot()
plt.title("Confusion Matrix (Val Data)")

In [ ]:
m_once = MLPModel()
train_model(m_once, train_data, val_data, learning_rate=0.5, batch_size=500, num_epochs=1)
print("Training Accuracy:", accuracy(m_once, train_data))
print("Validation Accuracy:", accuracy(m_once, val_data))

y, t = get_prediction(m_once, val_data)
y = y > 0.5
ConfusionMatrixDisplay(confusion_matrix(t, y), display_labels=["0", "1"]).plot()
plt.title("Confusion Matrix (Val Data)")